In [0]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta
import os
import warnings
import xlrd
import re
from string import punctuation
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
twt_sc_data=spark.table('relevant_twitter_supplychain_data_09_03')
twt_sc_data=twt_sc_data.toPandas()

twt_statedept_data=spark.table('twitter_statedept_data_07_03')
twt_statedept_data=twt_statedept_data.toPandas()

twt_general_news_data=spark.table('twitter_general_news_data')
twt_general_news_data=twt_general_news_data.toPandas()

twt_port_extreme_weather_data=spark.table('twitter_port_extreme_weather_data_analysis_07_03')
twt_port_extreme_weather_data=twt_port_extreme_weather_data.toPandas()

twt_port_traffic_congestion_data=spark.table('port_traffic_twitter_data_w_information_08_03')
twt_port_traffic_congestion_data=twt_port_traffic_congestion_data.toPandas()

twt_extreme_weather_data=spark.table('twitter_extreme_weather_data_04_03')
twt_extreme_weather_data=twt_extreme_weather_data.toPandas()

In [0]:
#Checking supply chain tweets data
twt_sc_data.head(5)

Out[40]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management


In [0]:
#Adding Classification column to classify tweet's category
twt_sc_data['Classification']='Supply Chain'
twt_sc_data.head(5)

Out[41]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain


In [0]:
#Checking port extreme weather data
twt_port_extreme_weather_data.head(5)

Out[42]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Cleaned_Text,Target_Date_Text,Tweet_Date,Target_Date
0,2020-11-08T22:26:47.000+0000,1325565457110343680,Port Everglades is closed as of 17:01 or 5:01...,PortEverglades,"Broward County, Florida",Port Everglades,port everglades is closed as of 17:01 or 5:01 ...,Sunday,2020-11-08,2020-11-08
1,2020-11-08T13:42:07.000+0000,1325433421179588608,@USCGSoutheast set Condition #Zulu for Tropica...,PortEverglades,"Broward County, Florida",Port Everglades,@uscgsoutheast set condition #zulu for tropica...,Sunday,2020-11-08,2020-11-08
2,2020-11-07T19:28:19.000+0000,1325158155861385218,@USCGSoutheast has set Condition Yankee for Tr...,PortEverglades,"Broward County, Florida",Port Everglades,@uscgsoutheast has set condition yankee for tr...,24 hours,2020-11-07,2020-11-07
3,2020-11-05T13:54:59.000+0000,1324349496650829824,The US Coast Guard Captain of the Port has imp...,PortEverglades,"Broward County, Florida",Port Everglades,the us coast guard captain of the port has imp...,72 hours,2020-11-05,2020-11-05
4,2021-02-10T22:36:57.000+0000,1359632475069108227,The @NWSSeattle has issued a winter storm watc...,PortofTacoma,"Tacoma, WA",Port of Tacoma,the @nwsseattle has issued a winter storm watc...,Thursday,2021-02-10,2021-02-11


In [0]:
#Removing unnecessary columns
twt_port_extreme_weather_data=twt_port_extreme_weather_data.drop(['Cleaned_Text','Target_Date_Text','Tweet_Date'],axis=1)
twt_port_extreme_weather_data.head(5)

Out[43]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Target_Date
0,2020-11-08T22:26:47.000+0000,1325565457110343680,Port Everglades is closed as of 17:01 or 5:01...,PortEverglades,"Broward County, Florida",Port Everglades,2020-11-08
1,2020-11-08T13:42:07.000+0000,1325433421179588608,@USCGSoutheast set Condition #Zulu for Tropica...,PortEverglades,"Broward County, Florida",Port Everglades,2020-11-08
2,2020-11-07T19:28:19.000+0000,1325158155861385218,@USCGSoutheast has set Condition Yankee for Tr...,PortEverglades,"Broward County, Florida",Port Everglades,2020-11-07
3,2020-11-05T13:54:59.000+0000,1324349496650829824,The US Coast Guard Captain of the Port has imp...,PortEverglades,"Broward County, Florida",Port Everglades,2020-11-05
4,2021-02-10T22:36:57.000+0000,1359632475069108227,The @NWSSeattle has issued a winter storm watc...,PortofTacoma,"Tacoma, WA",Port of Tacoma,2021-02-11


In [0]:
#Adding new column categorizing event in port data
new_col = 'Extreme Weather'
  
# Inserting the column before Traget date column
twt_port_extreme_weather_data.insert(loc = 6,
          column = 'Event',
          value = new_col)
# show the dataframe
twt_port_extreme_weather_data

Out[44]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Event,Target_Date
0,2020-11-08T22:26:47.000+0000,1325565457110343680,Port Everglades is closed as of 17:01 or 5:01...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-08
1,2020-11-08T13:42:07.000+0000,1325433421179588608,@USCGSoutheast set Condition #Zulu for Tropica...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-08
2,2020-11-07T19:28:19.000+0000,1325158155861385218,@USCGSoutheast has set Condition Yankee for Tr...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-07
3,2020-11-05T13:54:59.000+0000,1324349496650829824,The US Coast Guard Captain of the Port has imp...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-05
4,2021-02-10T22:36:57.000+0000,1359632475069108227,The @NWSSeattle has issued a winter storm watc...,PortofTacoma,"Tacoma, WA",Port of Tacoma,Extreme Weather,2021-02-11
...,...,...,...,...,...,...,...,...
59,2021-02-18T17:59:45.000+0000,1362461817767927808,"Friday 2/19, APMT, Maher Terminals &amp; PNCT...",PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Extreme Weather,2021-02-19
60,2021-02-17T17:49:45.000+0000,1362096913856815109,The Truck Service Center will be closed Thursd...,PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Extreme Weather,2021-02-18
61,2021-02-03T00:37:55.000+0000,1356763814931419141,"Due to continued snow removal efforts, PNCT wi...",PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Extreme Weather,2021-02-03
62,2021-02-01T19:38:15.000+0000,1356326013702205442,The Truck Service Center will be closed Tuesda...,PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Extreme Weather,2021-02-03


In [0]:
#Adding classification column
twt_port_extreme_weather_data['Classification']='Port'
twt_port_extreme_weather_data.head(5)

Out[45]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Event,Target_Date,Classification
0,2020-11-08T22:26:47.000+0000,1325565457110343680,Port Everglades is closed as of 17:01 or 5:01...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-08,Port
1,2020-11-08T13:42:07.000+0000,1325433421179588608,@USCGSoutheast set Condition #Zulu for Tropica...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-08,Port
2,2020-11-07T19:28:19.000+0000,1325158155861385218,@USCGSoutheast has set Condition Yankee for Tr...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-07,Port
3,2020-11-05T13:54:59.000+0000,1324349496650829824,The US Coast Guard Captain of the Port has imp...,PortEverglades,"Broward County, Florida",Port Everglades,Extreme Weather,2020-11-05,Port
4,2021-02-10T22:36:57.000+0000,1359632475069108227,The @NWSSeattle has issued a winter storm watc...,PortofTacoma,"Tacoma, WA",Port of Tacoma,Extreme Weather,2021-02-11,Port


In [0]:
#Creating new final file to append all the data
Final_Twitter_Analysis_Data=twt_sc_data.append(twt_port_extreme_weather_data)
Final_Twitter_Analysis_Data

Out[46]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN
...,...,...,...,...,...,...,...,...,...
59,2021-02-18T17:59:45.000+0000,1362461817767927808,"Friday 2/19, APMT, Maher Terminals &amp; PNCT...",PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Port,Extreme Weather,2021-02-19
60,2021-02-17T17:49:45.000+0000,1362096913856815109,The Truck Service Center will be closed Thursd...,PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Port,Extreme Weather,2021-02-18
61,2021-02-03T00:37:55.000+0000,1356763814931419141,"Due to continued snow removal efforts, PNCT wi...",PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Port,Extreme Weather,2021-02-03
62,2021-02-01T19:38:15.000+0000,1356326013702205442,The Truck Service Center will be closed Tuesda...,PortNYNJ,New York and New Jersey,Port of NY & NJ. Wear a Face Covering.,Port,Extreme Weather,2021-02-03


In [0]:
#Checking port traffic congestion data
twt_port_traffic_congestion_data.head(5)

Out[47]:

,Datetime,Tweet_Id,Text,Username,DisplayName,Cleaned_Text,target_traffic,target_port_congestion,Event,Cleaned_Text_List,Tweet_Date,Date_list,Target_Date_Text,Target_Date,tweet_year,Location
0,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,"weather conditions, today's covid testing port...",0.0,1.0,Port_Congestion,"weather conditions, today's covid testing port...",2020-12-08,[today],today,2020-12-08,2020,"Port Hueneme, CA"
1,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,"weather conditions, today's covid testing port...",0.0,1.0,Port_Congestion,*este evento ha sida cancelado*dec 8/martes 8 ...,2020-12-08,[covid-19],covid-19,2020-12-08,2020,"Port Hueneme, CA"
2,2020-03-16 23:34:42,1239696624902762496,The Port is open and operating as normal today...,portofhueneme,The Port of Hueneme,port open operating normal today. terminal coo...,0.0,1.0,Port_Congestion,port open operating normal today,2020-03-16,[today],today,2020-03-16,2020,"Port Hueneme, CA"
3,2020-08-04 11:07:12,1290605197660749828,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,continue monitor #isaias . seagirt marine term...,0.0,1.0,Port_Congestion,seagirt marine terminal ports america chesape...,2020-08-04,[today],today,2020-08-04,2020,"Baltimore, Maryland USA"
4,2020-08-03 20:58:03,1290391501869060097,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,continue monitor #isaias . seagirt marine term...,0.0,1.0,Port_Congestion,seagirt marine terminal ports america chesape...,2020-08-03,[tuesday],tuesday,2020-08-04,2020,"Baltimore, Maryland USA"


In [0]:
#Remove unnecessary columns
twt_port_traffic_congestion_data=twt_port_traffic_congestion_data.drop(['Cleaned_Text', 'target_traffic', 'target_port_congestion','Cleaned_Text_List', 'Tweet_Date', 'Date_list', 'Target_Date_Text','tweet_year','Location'],axis=1)
twt_port_traffic_congestion_data.head(5)

Out[48]:

,Datetime,Tweet_Id,Text,Username,DisplayName,Event,Target_Date
0,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,Port_Congestion,2020-12-08
1,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,Port_Congestion,2020-12-08
2,2020-03-16 23:34:42,1239696624902762496,The Port is open and operating as normal today...,portofhueneme,The Port of Hueneme,Port_Congestion,2020-03-16
3,2020-08-04 11:07:12,1290605197660749828,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,Port_Congestion,2020-08-04
4,2020-08-03 20:58:03,1290391501869060097,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,Port_Congestion,2020-08-04


In [0]:
#Adding classification column
twt_port_traffic_congestion_data['Classification']='Port'
twt_port_traffic_congestion_data.head()

Out[49]:

,Datetime,Tweet_Id,Text,Username,DisplayName,Event,Target_Date,Classification
0,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,Port_Congestion,2020-12-08,Port
1,2020-12-08 16:04:00,1336340763211309056,"DUE TO WEATHER CONDITIONS, TODAY'S COVID TESTI...",portofhueneme,The Port of Hueneme,Port_Congestion,2020-12-08,Port
2,2020-03-16 23:34:42,1239696624902762496,The Port is open and operating as normal today...,portofhueneme,The Port of Hueneme,Port_Congestion,2020-03-16,Port
3,2020-08-04 11:07:12,1290605197660749828,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,Port_Congestion,2020-08-04,Port
4,2020-08-03 20:58:03,1290391501869060097,We continue to monitor #Isaias . Seagirt Marin...,portofbalt,Port of Baltimore,Port_Congestion,2020-08-04,Port


In [0]:
#Appending to final table
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.append(twt_port_traffic_congestion_data)
Final_Twitter_Analysis_Data.head()

Out[50]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN


In [0]:
#Checking extreme weather data
twt_extreme_weather_data.head(5)

Out[51]:

,Datetime,Tweet_Id,Text,Username,Cleaned_Text,Cleaned_Text_List,Extreme_Weather_Event,Location,Target_Date_Text,Tweet_Date,Target_Date
0,2022-02-17 20:11:15,1494404095746224129,A broken band or bands of thunderstorms will c...,NWS,a broken band or bands of thunderstorms will c...,a broken band or bands of thunderstorms will c...,"storm,thunderstorm,tornado","alabama,mississippi",late this afternoon,2022-02-17,2022-02-17
1,2022-01-29 16:16:38,1487459680951672835,Heavy snow and blizzard conditions will contin...,NWS,heavy snow and blizzard conditions will contin...,heavy snow and blizzard conditions will contin...,"snow,heavy snow,blizzard","eastern ma,eastern long island",this afternoon,2022-01-29,2022-01-29
2,2021-12-22 02:39:56,1473483412962877444,A series of Pacific storms will bring periods ...,NWS,a series of pacific storms will bring periods ...,a storm strengthening off the east coast will...,"storm,snow,heavy snow",the east coast,Wednesday,2021-12-22,2021-12-22
3,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,snow is expected through sunday from the north...,snow is expected through sunday from the north...,"storm,snow",ohio valley,Sunday,2022-01-23,2022-01-23
4,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,snow is expected through sunday from the north...,a storm is forecast to shift across the gulf ...,"storm,heavy rain","mexico,florida",Tuesday,2022-01-23,2022-01-25


In [0]:
#Remove unnecessary columns
twt_extreme_weather_data=twt_extreme_weather_data.drop(['Cleaned_Text','Cleaned_Text_List','Target_Date_Text','Tweet_Date'],axis=1)

#Renaming location column
twt_extreme_weather_data=twt_extreme_weather_data.rename(columns={"Location":"Extreme_Weather_Location"})

twt_extreme_weather_data.head(5)

Out[52]:

,Datetime,Tweet_Id,Text,Username,Extreme_Weather_Event,Extreme_Weather_Location,Target_Date
0,2022-02-17 20:11:15,1494404095746224129,A broken band or bands of thunderstorms will c...,NWS,"storm,thunderstorm,tornado","alabama,mississippi",2022-02-17
1,2022-01-29 16:16:38,1487459680951672835,Heavy snow and blizzard conditions will contin...,NWS,"snow,heavy snow,blizzard","eastern ma,eastern long island",2022-01-29
2,2021-12-22 02:39:56,1473483412962877444,A series of Pacific storms will bring periods ...,NWS,"storm,snow,heavy snow",the east coast,2021-12-22
3,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,"storm,snow",ohio valley,2022-01-23
4,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,"storm,heavy rain","mexico,florida",2022-01-25


In [0]:
#Adding display name column as all above tables has this column
new_col1 = 'National Weather Service'
  
twt_extreme_weather_data.insert(loc = 4,
          column = 'DisplayName',
          value = new_col1)
twt_extreme_weather_data

Out[53]:

,Datetime,Tweet_Id,Text,Username,DisplayName,Extreme_Weather_Event,Extreme_Weather_Location,Target_Date
0,2022-02-17 20:11:15,1494404095746224129,A broken band or bands of thunderstorms will c...,NWS,National Weather Service,"storm,thunderstorm,tornado","alabama,mississippi",2022-02-17
1,2022-01-29 16:16:38,1487459680951672835,Heavy snow and blizzard conditions will contin...,NWS,National Weather Service,"snow,heavy snow,blizzard","eastern ma,eastern long island",2022-01-29
2,2021-12-22 02:39:56,1473483412962877444,A series of Pacific storms will bring periods ...,NWS,National Weather Service,"storm,snow,heavy snow",the east coast,2021-12-22
3,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,National Weather Service,"storm,snow",ohio valley,2022-01-23
4,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,National Weather Service,"storm,heavy rain","mexico,florida",2022-01-25
5,2022-02-02 02:00:25,1488693759013535747,Moderate to heavy snow appears likely to devel...,NWS,National Weather Service,"snow,heavy snow","west central,illinois,missouri",2022-02-02
6,2021-12-14 20:16:09,1470850114633801732,"Tomorrow (Wednesday), a band of strong storms ...",NWS,National Weather Service,storm,missouri,2021-12-15
7,2022-01-24 01:48:35,1485429291001151491,A second clipper will cross the northern Plain...,NWS,National Weather Service,"storm,heavy rain",florida,2022-01-25
8,2022-01-21 14:12:40,1484529382752464909,A strong to severe thunderstorm or two may occ...,NWS,National Weather Service,"storm,hail,thunderstorm",florida,2022-01-21
9,2022-02-24 02:54:51,1496679991999795203,A significant winter storm will bring snow and...,NWS,National Weather Service,"storm,snow,heavy snow",the ohio valley,2022-02-24


In [0]:
#Adding classification column
twt_extreme_weather_data['Classification']='Extreme Weather'
twt_extreme_weather_data.head(5)

Out[54]:

,Datetime,Tweet_Id,Text,Username,DisplayName,Extreme_Weather_Event,Extreme_Weather_Location,Target_Date,Classification
0,2022-02-17 20:11:15,1494404095746224129,A broken band or bands of thunderstorms will c...,NWS,National Weather Service,"storm,thunderstorm,tornado","alabama,mississippi",2022-02-17,Extreme Weather
1,2022-01-29 16:16:38,1487459680951672835,Heavy snow and blizzard conditions will contin...,NWS,National Weather Service,"snow,heavy snow,blizzard","eastern ma,eastern long island",2022-01-29,Extreme Weather
2,2021-12-22 02:39:56,1473483412962877444,A series of Pacific storms will bring periods ...,NWS,National Weather Service,"storm,snow,heavy snow",the east coast,2021-12-22,Extreme Weather
3,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,National Weather Service,"storm,snow",ohio valley,2022-01-23,Extreme Weather
4,2022-01-23 02:55:33,1485083754389336065,Snow is expected through Sunday from the north...,NWS,National Weather Service,"storm,heavy rain","mexico,florida",2022-01-25,Extreme Weather


In [0]:
#Appending to final data
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.append(twt_extreme_weather_data)
Final_Twitter_Analysis_Data.head()

Out[55]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date,Extreme_Weather_Event,Extreme_Weather_Location
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN


In [0]:
#Checking state dept tweets data
twt_statedept_data.head(5)

Out[56]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification
0,2022-01-16 01:30:00,1482525509925941250,.@JakeSullivan46: Our position is quite straig...,StateDept,"Washington, DC",Department of State,State Department News
1,2022-01-16 00:00:00,1482502860591927296,.@JakeSullivan46: We have been very clear with...,StateDept,"Washington, DC",Department of State,State Department News
2,2022-01-15 22:00:00,1482472662148554758,.@SecBlinken: “What’s happening right now with...,StateDept,"Washington, DC",Department of State,State Department News
3,2022-01-15 21:00:00,1482457563681615872,The U.S. delivered nearly 3 million additiona...,StateDept,"Washington, DC",Department of State,State Department News
4,2022-01-15 20:30:00,1482450012596424705,.@StateEUR’s Assistant Secretary Donfried on d...,StateDept,"Washington, DC",Department of State,State Department News


In [0]:
#Creating a new dataframe and appending data
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.append(twt_statedept_data)
Final_Twitter_Analysis_Data

Out[57]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date,Extreme_Weather_Event,Extreme_Weather_Location
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
774,2022-02-24 00:23:08,1496641809879085062,Our Allies and partners are united in our reso...,StateDept,"Washington, DC",Department of State,State Department News,NaN,NaN,NaN,NaN
775,2022-02-23 21:15:00,1496594463933669381,.@StateDeptSpox: Where do we go from here? Mos...,StateDept,"Washington, DC",Department of State,State Department News,NaN,NaN,NaN,NaN
776,2022-02-23 20:27:16,1496582453049319425,#UnitedWithUkraine https://t.co/flzqwod2uF,StateDept,"Washington, DC",Department of State,State Department News,NaN,NaN,NaN,NaN
777,2022-02-23 19:10:17,1496563079538257921,Live now! @StateDeptSpox Ned Price briefs repo...,StateDept,"Washington, DC",Department of State,State Department News,NaN,NaN,NaN,NaN


In [0]:
#Checking state dept tweets data
twt_general_news_data.head(5)

Out[58]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification
0,2022-02-16 11:10:02,1493905505894506506,Knowing the intent of any autocratic leader is...,nytimes,New York City,The New York Times,General News
1,2022-02-16 11:00:05,1493903002905858058,American Olympians won both the gold and silve...,nytimes,New York City,The New York Times,General News
2,2022-02-16 10:50:06,1493900486780624901,At least 10 people are dead and more than a do...,nytimes,New York City,The New York Times,General News
3,2022-02-16 10:30:11,1493895478571843586,Prince Andrew’s settlement with Virginia Giuff...,nytimes,New York City,The New York Times,General News
4,2022-02-16 10:00:11,1493887928145219589,"In Opinion\n\n""To an ethnic Russian who came o...",nytimes,New York City,The New York Times,General News


In [0]:
#Creating a new dataframe and appending data
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.append(twt_general_news_data)
Final_Twitter_Analysis_Data.head(5)

Out[59]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date,Extreme_Weather_Event,Extreme_Weather_Location
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN


In [0]:
#Resetting the index of final data
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.reset_index(drop=True)
Final_Twitter_Analysis_Data.head()

Out[60]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date,Extreme_Weather_Event,Extreme_Weather_Location
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN


In [0]:
Final_Twitter_Analysis_Data.shape

Out[61]: (58676, 11)

In [0]:
#Lower case conversion of text column
Final_Twitter_Analysis_Data['Cleaned_Text']=Final_Twitter_Analysis_Data['Text']

#Removing URLs
def remove_urls(text):
	url_pattern = r'https?://\S+|www\.\S+'
	without_urls = re.sub(pattern=url_pattern, repl='', string=text)
	return without_urls
  
#Removing Emojis
def remove_emojis(text):
	emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

	without_emoji = emoji_pattern.sub(r'',text)
	return without_emoji

#Cleaning unwanted texts
Final_Twitter_Analysis_Data['Cleaned_Text'].replace(regex=True, to_replace =['¢','€', 'Ã', '¬', 'Ð', '±','½','©','•','¾', '§', '¥', '«', '¤', '–', 'œ', '¡', '”', '|', 'â', '™', 'Â', 'Î', '¿', 'µ', '´', '‡','»', 'Ž', '®', 'º', 'Ï', 'ƒ', '¶', '¹', '┬', 'á', 'Γ', 'Ç', 'Ö','Ë','œ','Ã','£','ð','Ÿ','š','ª','²','Ã','£','°','¿','Â','ž','&amp;','&amp','â','ã','~','J&amp;J','','˜','ÿ’','›','†','ðŸ’™ðŸ†ðŸ’™ðŸ†ðŸ’™','ÿ','“','Ÿ‘','Ÿ’','Ÿ','ï','—','ï','ï','ÿ‘','ÿ—ï','ÿ—“ï','³','ÿ“','·:','‹','ÿ‘','ÿ˜','¨','‰','¸'], value='', inplace=True)

def clean_data(text):
    text=text.replace('âš ï','')
    text=text.replace('âœ','')
    text=text.replace('&amp;','')
    text=text.replace('â€™','')
    text=text.replace('â€œ','')
    text=text.replace('â€','')
    text=text.replace('ðŸ”¥ðŸš’','')
    text=text.replace('â„ï¸','')
    text=text.replace('â€¼ï¸','')
    text=text.replace('ðŸŽµðŸŽ¶','')
    text=text.replace('ðÿž¶ðÿžµ','')
    text=text.replace('ðÿžµðÿž¶','')
    text=text.replace('ðŸŽ¶ðŸŽµ','')
    text=text.replace('ðŸŽ¶ðŸª•','')
    text=text.replace('ªð','')
    text=text.replace('²ð','')
    text=text.replace('µð','')
    text=text.replace('¹ð','')
    text=text.replace('ºð','')
    text=text.replace('ºðÿ','')
    text=text.replace('¼ð','')
    text=text.replace('½ðÿš','')
    text=text.replace('ðÿ','')
    text=text.replace('ðÿœš','')
    text=text.replace('ðÿš','')
    text=text.replace('ðÿž','')
    text=text.replace('\n','')
    for char in ['²the', '³ð', 'áÿ', 'áÿƒáž', 'áž', 'áž¼ážšážˆáÿ', 'áž½áž', 'áž¾ážˆáÿ', 'ážÿáÿ', 'ážÿáž', 'ážšáž', 'ážšáž¹áž', 'ážšáž¼ážœáž', 'ážšáž¾áž', 'âš', 'âž', 'žƒ', 'ƒï']:
      text=text.replace('char','')
    return text

#Removing Extra Whitespaces
def remove_extra_spaces(text):
	space_pattern = r'\s+'
	without_space = re.sub(pattern=space_pattern, repl=" ", string=text)
	return without_space
  
#Removing punctuations using string library
def remove_punctuation(text):
	return text.translate(str.maketrans('', '', punctuation))
  

#Applying function
Final_Twitter_Analysis_Data['Cleaned_Text'] = Final_Twitter_Analysis_Data['Cleaned_Text'].apply(remove_urls)
Final_Twitter_Analysis_Data['Cleaned_Text'] = Final_Twitter_Analysis_Data['Cleaned_Text'].apply(remove_emojis)
Final_Twitter_Analysis_Data['Cleaned_Text'] = Final_Twitter_Analysis_Data['Cleaned_Text'].apply(clean_data)
Final_Twitter_Analysis_Data['Cleaned_Text'] = Final_Twitter_Analysis_Data['Cleaned_Text'].apply(remove_extra_spaces)

In [0]:
#Creating hyperlink column with tweet URLs in list
import numpy as np
Final_Twitter_Analysis_Data['URLS_data']=np.nan
def adding_urls(text):
  url_pattern = r'https?://\S+|www\.\S+'
  with_urls = re.findall(url_pattern, text)
  return with_urls

for i in range(Final_Twitter_Analysis_Data.shape[0]):
  urls=Final_Twitter_Analysis_Data['Text'].iloc[i]
  urls_result = adding_urls(urls)
  Final_Twitter_Analysis_Data['URLS_data'].iloc[i]=urls_result

/databricks/python/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 self._setitem_single_block(indexer, value, name)

In [0]:
#Getting all the urls of a tweet separated by ","
Final_Twitter_Analysis_Data['Hyperlink_Url']=np.nan
for urls, indices in zip(Final_Twitter_Analysis_Data['URLS_data'],Final_Twitter_Analysis_Data.index):
  if len(urls)>0:
    b=str(urls)
    c=b[1:-1]
    c=c.replace('"','')
    c=c.replace("'","")
    Final_Twitter_Analysis_Data['Hyperlink_Url'][indices]=c
  else:
    c=""
    Final_Twitter_Analysis_Data['Hyperlink_Url'][indices]=c

<command-971031596573397>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 Final_Twitter_Analysis_Data['Hyperlink_Url'][indices]=c

In [0]:
#Removing Url data column
Final_Twitter_Analysis_Data=Final_Twitter_Analysis_Data.drop(['URLS_data'],axis=1)
Final_Twitter_Analysis_Data.head()

Out[65]:

,Datetime,Tweet_Id,Text,Username,UserLocation,DisplayName,Classification,Event,Target_Date,Extreme_Weather_Event,Extreme_Weather_Location,Cleaned_Text,Hyperlink_Url
0,2020-04-16T16:52:35.000Z,1250829450956199936,POLB and POLA see further volume declines in M...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN,POLB and POLA see further volume declines in M...,https://t.co/0ml95nVibO
1,2020-03-16T18:33:47.000Z,1239620896857469952,San Francisco Bar Pilots solve one Pacific Rim...,LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN,San Francisco Bar Pilots solve one Pacific Rim...,"https://t.co/LHUevvsZ8M, https://t.co/wkpmXLnVkw"
2,2020-02-20T16:52:31.000Z,1230535714930139904,"#Freight TSI falls for second straight month, ...",LogisticsMgmt,"Framingham, MA",Logistics Management,Supply Chain,NaN,NaN,NaN,NaN,"#Freight TSI falls for second straight month, ...","https://t.co/AUOBWGal9h, https://t.co/S4BtCmD2dU"
3,2021-10-27T21:17:32.000Z,1453470946572910080,"It's not you, it's...the backlog of ships in t...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN,"It's not you, it's...the backlog of ships in t...",https://t.co/TtyQnKwy75
4,2021-10-14T19:26:13.000Z,1448731890358240000,"""U.S. global imports has increased by 20% (est...",uclaanderson,"Los Angeles, CA",UCLA Anderson School of Management,Supply Chain,NaN,NaN,NaN,NaN,"""U.S. global imports has increased by 20% (est...",https://t.co/7oah7LKEEr


In [0]:
#Checking classification of data:
Final_Twitter_Analysis_Data['Classification'].unique()

Out[66]: array(['Supply Chain', 'Port', 'Extreme Weather', 'State Department News',
 'General News'], dtype=object)

In [0]:
Final_Twitter_Analysis_Data['Event']=Final_Twitter_Analysis_Data['Event'].astype(str)
Final_Twitter_Analysis_Data['Target_Date']=Final_Twitter_Analysis_Data['Target_Date'].astype(str)
Final_Twitter_Analysis_Data['Datetime']=Final_Twitter_Analysis_Data['Datetime'].astype(str)
Final_Twitter_Analysis_Data['Tweet_Id']=Final_Twitter_Analysis_Data['Tweet_Id'].astype(str)


In [0]:
#Saving the data
twitter_data=spark.createDataFrame(Final_Twitter_Analysis_Data)
twitter_data.write.saveAsTable('3_Twitter_data', mode = 'overwrite')